# Representação de textos: Embeddings (Word2Vec)

*   Sklearn -> https://www.youtube.com/watch?v=bwqBZ4IuX7Q
*   Pratical NLP Book -> https://github.com/practical-nlp/practical-nlp-code/tree/master/Ch3
* Vídeo: https://www.youtube.com/watch?v=Do8cVbx-HOs&list=PLeo1K3hjS3uuvuAXhYjV2lMEShq2UYSwX&index=19




In [1]:
!pip3 install scikit-learn

In [1]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 1.0 MB/s eta 0:00:000:00:01m eta 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 331.1 kB/s eta 0:00:00m eta 0:00:010:00:03
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [scipy]
      Successfully uninstalled scipy-1.15.2━━━━━━━ 0/2 [scipy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [gensim]━━━━ 1/2 [gensim]


### Exercícios de aplicação com Word2Vec (inglês)


Link: https://towardsdatascience.com/word-embedding-techniques-word2vec-and-tf-idf-explained-c5d02e34d08

Dados: https://www.kaggle.com/datasets/anu0012/hotel-review

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/grati/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
model = KeyedVectors.load_word2vec_format('cbow_s50.txt', binary=False)

In [4]:
import pandas as pd
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import multiprocessing

csv_path = "data/train.csv"

df = pd.read_csv(csv_path, on_bad_lines='warn')

def clean_data(text):
    text = re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ/]+', '', text)
    text = re.sub(r'[\\/×\^\]\[÷]', '', text)
    return text

def change_lower(text):
    text = text.lower()
    return text

stopwords_list = stopwords.words("english")

def remover(text):
    text_tokens = text.split(" ")
    final_list = [word for word in text_tokens if not word in stopwords_list]
    text = ' '.join(final_list)
    return text

def get_w2vdf(df):
    w2v_df = pd.DataFrame(df["Description"]).values.tolist()
    for i in range(len(w2v_df)):
        w2v_df[i] = w2v_df[i][0].split(" ")
    return w2v_df

def train_w2v(w2v_df):
    cores = multiprocessing.cpu_count()
    w2v_model = Word2Vec(min_count=4,
                         window=4,
                         vector_size=300,
                         alpha=0.03,
                         min_alpha=0.0007,
                         sg = 1,
                         workers=cores-1)

    w2v_model.build_vocab(w2v_df, progress_per=10000)
    w2v_model.train(w2v_df, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)
    return w2v_model

In [ ]:
# df[["reviews.text"]] = df[["reviews.text"]].astype(str)
# df["reviews.text"] = df["reviews.text"].apply(change_lower)
# df["reviews.text"] = df["reviews.text"].apply(clean_data)
# df["reviews.text"] = df["reviews.text"].apply(remover)

df[["Description"]] = df[["Description"]].astype(str)
df["Description"] = df["Description"].apply(change_lower)
df["Description"] = df["Description"].apply(clean_data)
df["Description"] = df["Description"].apply(remover)

w2v_df = get_w2vdf(df)
w2v_model = train_w2v(w2v_df)

In [ ]:
w2v_model.wv.most_similar(positive=["great"])

In [ ]:
w2v_model.wv.most_similar(positive=["terrible"])

### Exercícios de aplicação com Word2Vec (espanhol)
Link: https://keepcoding.io/blog/ejercicio-de-aplicacion-con-word2vec/

Dados: https://www.kaggle.com/datasets/kevinmorgado/spanish-news-classification

In [ ]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

In [ ]:
corpus = LineSentence('spanish-news.csv', limit = 1000)

In [ ]:
sg_params = {
          'vector_size' : 20,
          'window' : 10,
          'min_count' : 5,
          'sg' : 1,
          'hs' : 0,
          'negative' : 20,
}

cbow_params = {
          'vector_size' : 20,
          'window' : 10,
          'min_count' : 5,
          'sg' : 0,
          'hs' : 0,
          'negative' : 20,
}

In [ ]:
#Skip Gram
w2v_sg = Word2Vec(** sg_params)

#CBOW
w2v_cbow = Word2Vec(** cbow_params)

In [ ]:
#Skip Gram
w2v_sg.build_vocab(corpus)

#CBOW
w2v_cbow.build_vocab(corpus)

In [ ]:
#Skip Gram
w2v_sg.train(corpus_iterable = corpus, total_examples = w2v_sg.corpus_count, epochs = 10)

(685700, 1306180)

In [ ]:
#CBOW
w2v_cbow.train(corpus_iterable = corpus, total_examples = w2v_cbow.corpus_count, epochs = 10)

(686208, 1306180)

In [ ]:
w2v_sg.save('./w2v_sg_ d300_mc5_ w10.pkl')
w2v_cbow.save('./w2v_sg_ d300_mc5_ w10.pkl')

In [ ]:
def print_sim_words(word, model_cbow, model_sg):
    """Prints similar words for a given word using both CBOW and Skip-Gram models."""

    print(f"Similar words to '{word}' (CBOW):")
    for similar_word, similarity in model_cbow.wv.most_similar(word):
        print(f"  {similar_word}: {similarity:.4f}")

    print(f"\nSimilar words to '{word}' (Skip-Gram):")
    for similar_word, similarity in model_sg.wv.most_similar(word):
        print(f"  {similar_word}: {similarity:.4f}")

    print("-" * 30)

In [ ]:
print_sim_words('elecciones', w2v_cbow, w2v_sg)

Similar words to 'elecciones' (CBOW):
  fuera: 0.9963
  crecieron: 0.9962
  doble: 0.9955
  tuvieron: 0.9952
  estadística: 0.9952
  aumentando: 0.9952
  Turquía: 0.9951
  prevén: 0.9950
  FOMC: 0.9949
  Central: 0.9948

Similar words to 'elecciones' (Skip-Gram):
  estadounidense: 0.9692
  ajustes: 0.9684
  virus: 0.9617
  semanas: 0.9582
  precios.: 0.9498
  pronósticos: 0.9497
  probablemente: 0.9465
  causas: 0.9447
  recesión: 0.9421
  interno: 0.9414
------------------------------


### Comparando Word2Vec, Glove e FastText
Link: https://medium.com/@mervebdurna/advanced-word-embeddings-word2vec-glove-and-fasttext-26e546ffedbd

In [ ]:
!pip install glove-python3

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Code Example with Toy Dataset
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# Toy dataset
sentences = ["I love natural language processing.",
             "Word embeddings are powerful."]

# Tokenize sentences
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Access embeddings
word_embeddings = model.wv
print(word_embeddings['natural'])

[ 8.1681199e-03 -4.4430327e-03  8.9854337e-03  8.2536647e-03
 -4.4352221e-03  3.0310510e-04  4.2744912e-03 -3.9263200e-03
 -5.5599655e-03 -6.5123225e-03 -6.7073823e-04 -2.9592158e-04
  4.4630850e-03 -2.4740540e-03 -1.7260908e-04  2.4618758e-03
  4.8675989e-03 -3.0808449e-05 -6.3394094e-03 -9.2608072e-03
  2.6657581e-05  6.6618943e-03  1.4660227e-03 -8.9665223e-03
 -7.9386048e-03  6.5519023e-03 -3.7856805e-03  6.2549924e-03
 -6.6810320e-03  8.4796622e-03 -6.5163244e-03  3.2880199e-03
 -1.0569858e-03 -6.7875278e-03 -3.2875966e-03 -1.1614120e-03
 -5.4709399e-03 -1.2113475e-03 -7.5633135e-03  2.6466595e-03
  9.0701487e-03 -2.3772502e-03 -9.7651005e-04  3.5135616e-03
  8.6650876e-03 -5.9218528e-03 -6.8875779e-03 -2.9329848e-03
  9.1476962e-03  8.6626766e-04 -8.6784009e-03 -1.4469790e-03
  9.4794659e-03 -7.5494875e-03 -5.3580985e-03  9.3165627e-03
 -8.9737261e-03  3.8259076e-03  6.6544057e-04  6.6607012e-03
  8.3127534e-03 -2.8507852e-03 -3.9923131e-03  8.8979173e-03
  2.0896459e-03  6.24894

In [ ]:
from glove import Corpus, Glove
from nltk.tokenize import word_tokenize

# Toy dataset
sentences = ["Word embeddings capture semantic meanings.",
             "GloVe is an impactful word embedding model."]

# Tokenize sentences
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]

# Creating a corpus object
corpus = Corpus()

# Training the corpus to generate the co-occurrence matrix
corpus.fit(tokenized_sentences, window=10)

# Training the GloVe model
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

# Retrieve and display word embeddings
word = "glove"
embedding = glove.word_vectors[glove.dictionary[word]]
print(f"Embedding for '{word}': {embedding}")

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Embedding for 'glove': [-1.10675337e-03 -3.24429905e-03 -3.58508580e-03 -4.14058402e-03
 -2.01957304e-03 -3.33836085e-03  2.13817485e-03 -5.78321509e-04
  2.02532813e-03  6.04360047e-04  3.77675097e-03 -2.78383440e-04
 -4.77656701e-04  2.45030037e-03  3.45739369e-03 -2.88659816e-03
  4.40631336e-03 -6.03074771e-04  3.33246233e-03  3.26901194e-03
  2.88213702e-03  4.28857362e-03  1.79638300e-03  3.79029344e-03
  5.02457618e-03 -4.15062544e-03 -3.28398403e-03 -4.36928350e-03
 -3.89783777e-03 -1.04070763e-03 -6.67592954e-04  2.89479569e-03
  2.25368724e-04  4.16515835e-03  4.72576193e-03 -3.65519017e-04
 -4.69906826e-03  4.83716493e-04 -4.08917522e-03 -3.49092671e-03
  2.62931448e-03 -4.81

In [ ]:
from gensim.models import FastText
from nltk.tokenize import word_tokenize

# Toy dataset
sentences = ["FastText embeddings handle subword information.",
             "It is effective for various languages."]
# Tokenize sentences
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]

# Train FastText model
model = FastText(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Access embeddings
word_embeddings = model.wv
print(word_embeddings['subword'])

[ 1.2919701e-03 -1.0602611e-04 -1.1323356e-03  1.6584302e-03
 -5.7117449e-04 -2.9840841e-04 -4.3193492e-04 -3.1250282e-04
 -1.9898350e-04  9.4852143e-04  1.6994212e-03 -1.8581563e-04
 -8.1228669e-04 -1.5968895e-03 -1.3839703e-03 -6.3576088e-05
 -5.7436171e-04 -1.1720147e-03  7.4763177e-04 -2.1684753e-05
  4.5981101e-04 -1.7291495e-03 -4.4365969e-04  4.0478929e-04
  1.2072949e-04  6.4071972e-04 -1.0785459e-03  1.3050955e-03
  3.5044085e-04 -1.8284899e-04 -1.5951110e-04 -1.0465594e-03
 -1.5170674e-04 -5.7858619e-04 -1.8307484e-03  1.0248278e-03
  6.9344341e-04  1.6159177e-03 -8.4400486e-04  8.9535897e-04
 -1.3508157e-04  2.3538095e-03 -3.7109022e-04 -2.8064058e-04
  2.6269807e-04 -2.8326022e-04 -7.7332847e-04  1.8949938e-03
  2.1798143e-03 -4.4569728e-04 -6.4175081e-04  1.4240020e-04
  2.5182988e-03 -1.5666584e-03  1.3954224e-04 -6.9046958e-04
  5.8793183e-04 -1.4282564e-03  2.1278318e-04 -2.2993281e-03
 -4.3249400e-03 -1.6397990e-03  1.3989839e-03 -1.3229308e-03
  2.0258871e-03 -2.96638